Instalamos los paquetes necesarios para trabajar con Spark en Google Colab.

- `pyspark`: permite utilizar Apache Spark desde Python.

- `findspark`: facilita la integración de Spark con notebooks al detectar automáticamente la instalación de Spark.

- La opción `-q` silencia la salida para que el proceso sea más limpio.

In [1]:
!pip install pyspark findspark -q

Descargamos el archivo `.jar` del conector `JDBC` de `MySQL`, necesario para que `PySpark` pueda conectarse a bases de datos `MySQL`.
El archivo contiene las clases que permiten establecer la conexión entre Spark y `MySQL` usando `JDBC`.
La opción `-q` se usa para suprimir la salida del proceso de descarga.

In [2]:
!wget https://repo.maven.apache.org/maven2/mysql/mysql-connector-java/8.0.30/mysql-connector-java-8.0.30.jar -q

Inicializamos `PySpark` dentro del entorno de ejecución de Colab.
La librería `findspark` configura las variables de entorno necesarias para que `Python` pueda encontrar e importar los módulos de Spark correctamente.

In [3]:
import findspark

findspark.init()

Importamos los módulos necesarios para trabajar con PySpark y leer datos:

- `SparkSession` nos permite iniciar una sesión de Spark.

- `col` se usa para manipular columnas en transformaciones de `DataFrames`.

- `json` nos permite cargar las credenciales almacenadas en un archivo `.json`.

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import json

Creamos una sesión de `Spark` y le indicamos la ruta al conector `JDBC` de `MySQL` mediante `.config("spark.jars", ...)`. Esto permite establecer conexiones entre `PySpark` y una base de datos `MySQL`.

In [5]:
spark = SparkSession.builder \
    .appName('PySpark MySQL Connection') \
    .config('spark.jars', 'mysql-connector-java-8.0.30.jar') \
    .getOrCreate()

print('Spark configurado con el conector de MySQL.')

Spark configurado con el conector de MySQL.


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Definimos la ruta al archivo `credentials.json`, que contiene las credenciales necesarias (como usuario, contraseña, host y base de datos) para conectarnos de forma segura a `MySQL` desde `PySpark`.

In [8]:
credentials_file = '/content/drive/MyDrive/TSCDeIA - Práctica Profesionalizante/Procesamiento de Datos/credentials.json'

Abrimos el archivo `credentials.json` y cargamos su contenido como un diccionario usando `json.load`. Luego, extraemos los datos de conexión a la base de datos (host, puerto, usuario, contraseña, nombre de base y tabla) utilizando el método `.get()` para evitar errores si alguna clave no existe. Esto nos permite mantener las credenciales fuera del código fuente y trabajar de forma más segura.

In [9]:
with open(credentials_file, 'r') as f:
  credentials = json.load(f)

db_host = credentials.get('db_host')
db_port = credentials.get('db_port')
db_user = credentials.get('db_user')
db_password = credentials.get('db_password')
db_name = credentials.get('db_name')
db_table = credentials.get('db_table')

print('Credenciales cargadas desde el archivo credentials.json.')

Credenciales cargadas desde el archivo credentials.json.


A partir de las credenciales cargadas, construimos la `URL` de conexión `JDBC` que `Spark` usará para acceder a la base de datos `MySQL`. Esta `URL` incluye el host, puerto y nombre de la base, además de un parámetro adicional (zeroDateTimeBehavior=round) que permite manejar valores de fecha nula en `MySQL` sin generar errores.

In [ ]:
jdbc_url = f"jdbc:mysql://{db_host}:{db_port}/{db_name}?zeroDateTimeBehavior=round"

print('URL contruida a partir de las credenciales corretamente')

Este bloque de código verifica que todas las variables necesarias estén definidas y que `db_table` tenga un valor válido. Si es así, intenta conectarse a la base de datos `MySQL` mediante el conector `JDBC` y cargar la tabla especificada dentro de un `DataFrame` de `Spark`. Esta operación nos permite integrar datos relacionales con el ecosistema distribuido de `PySpark`, facilitando su procesamiento posterior. Además, se maneja cualquier error que pueda surgir durante la conexión o la lectura, mostrando un mensaje claro para depuración.

In [ ]:
if 'jdbc_url' in locals() and 'db_user' in locals() and 'db_password' in locals() and 'db_table' in locals() and db_table is not None:
    try:
        print(f"Intentando leer la tabla '{db_table}' de la base de datos MySQL...")
        data = spark.read.format('jdbc') \
            .option('url', jdbc_url) \
            .option('driver', 'com.mysql.cj.jdbc.Driver') \
            .option('dbtable', db_table) \
            .option('user', db_user) \
            .option('password', db_password) \
            .load()

        print('Datos leídos exitosamente en un DataFrame de Spark.')
    except Exception as e:
        print(f'Ocurrió un error al intentar leer la tabla: {e}')
else:
    print("Error: Las credenciales o el nombre de la tabla no están disponibles o 'db_table' es None.")
    print("Por favor, asegúrate de ejecutar la celda anterior para cargar las credenciales y verificar que 'db_table' esté presente y tenga un valor en credentials.json.")

Este bloque nos permite visualizar la estructura del `DataFrame` cargado desde la base de datos. Al utilizar `printSchema()`, `PySpark` muestra los nombres de las columnas, sus tipos de datos y si permiten valores nulos. Esta información es clave para validar que los datos se hayan leído correctamente y que la inferencia de tipos sea coherente con el esquema de la base original. Nos facilita además planificar los pasos posteriores de transformación o limpieza.

In [ ]:
print(f'Esquema del DataFrame:')
data.printSchema()

Utilizamos la función `show()` de un `DataFrame` de `Spark` para visualizar las primeras filas de los datos cargados. Primero, imprimimos un mensaje informativo para indicar que se mostrarán las primeras filas. Luego, con `data.show()`, se despliegan por defecto las primeras 20 filas del `DataFrame` en formato tabular, lo que permite una inspección rápida del contenido y la estructura de los datos.

In [ ]:
print('Mostrando las primeras filas del DataFrame:')
data.show()

Obtenemos una lista con los nombres de todas las columnas del `DataFrame` original `data`. La propiedad `.columns` devuelve una lista de strings que representa cada nombre de columna, lo cual es útil para conocer la estructura del `DataFrame` y poder manipular o analizar las columnas de manera programática.

In [ ]:
columns = data.columns

Definimos una lista llamada `excluded_columns` que contiene los nombres de las columnas que queremos excluir del análisis o procesamiento posterior. Esta lista incluye campos relacionados con datos personales, metadatos de auditoría y observaciones, que pueden no ser relevantes o necesarias para ciertas operaciones de limpieza, modelado o visualización.

In [ ]:
excluded_columns = ["archivo", "apellido", "nombre", "dni", "updated_at", "deleted_at", "created_by", "updated_by", "deleted_by", "observaciones", "resultado_laboratorio"]

Creamos una nueva lista llamada `selected_columns` que contiene todas las columnas originales excepto las que están en la lista `excluded_columns`. Para esto, usamos una comprensión de listas que filtra las columnas que no están en la lista de exclusión. Luego, generamos un nuevo `DataFrame` data seleccionando únicamente las columnas de `selected_columns` mediante el método `.select()`, lo que permite trabajar con un subconjunto de columnas relevante para nuestro análisis o procesamiento.

In [ ]:
selected_columns = [col for col in columns if col not in excluded_columns]

data = data.select(*selected_columns)

Imprimimos en consola el esquema del nuevo `DataFrame` data para verificar la estructura y tipos de datos de las columnas seleccionadas. Primero mostramos un mensaje informativo y luego usamos el método `printSchema()` de `Spark`, que despliega una representación jerárquica con los nombres de las columnas y sus tipos de datos, facilitando la inspección rápida del `DataFrame`.

In [ ]:
print("Esquema del DataFrame con columnas seleccionadas:")
data.printSchema()

Mostramos en consola un mensaje informativo y luego utilizamos el método `show()` del `DataFrame` data para visualizar las primeras filas con las columnas seleccionadas. Esto permite verificar rápidamente el contenido y confirmar que la selección de columnas se realizó correctamente.

In [ ]:
print('Mostrando las primeras filas del DataFrame con columnas seleccionadas:')
data.show()

Definimos la columna `establecimiento_notificador` como la que utilizaremos para aplicar un filtro en el DataFrame. Luego construimos una condición de filtrado que verifica que los valores de esa columna no sean nulos `(isNotNull())` y que además no sean cadenas vacías (`!= ""`). Esta condición permite limpiar los datos para trabajar solo con registros que tengan un valor válido en la columna especificada.

In [ ]:
filter_column = "establecimiento_notificador"

filter_selected = col(filter_column).isNotNull() & (col(filter_column) != "")

Aplicamos un filtro al `DataFrame` data utilizando la condición almacenada en `filter_selected`, generando un nuevo `DataFrame` llamado `data_v2`, que contiene solo las filas que cumplen con dicho filtro. Luego, imprimimos en consola la cantidad de registros antes y después del filtrado, para así poder comparar y verificar cuántos registros fueron retenidos tras aplicar la condición de filtro.

In [ ]:
data_v2 = data.filter(filter_selected)

print(f"Número de registros antes del filtrado por '{filter_column}': {data.count()}")
print(f"Número de registros después del filtrado por '{filter_column}': {data_v2.count()}")

Mostramos las primeras filas del `DataFrame` filtrado según la columna especificada en `filter_column`. A continuación, con el método `.show()` mostramos esas filas para verificar visualmente el resultado del filtrado aplicado.

In [ ]:
print(f"Mostrando las primeras filas del DataFrame filtrado por '{filter_column}':")
data_v2.show()

Convertimos el `DataFrame` de `Spark` `data_v2` a un `DataFrame` de `pandas` utilizando el método `toPandas()`. Esta conversión permite trabajar con las herramientas y funciones de `pandas`, que son muy usadas para análisis y visualización en memoria. Si la conversión es exitosa, imprimimos un mensaje de confirmación, mostramos las primeras filas con `display()` y también la cantidad total de registros en el `DataFrame` de `pandas`. En caso de que ocurra algún error durante la conversión, lo capturamos e imprimimos un mensaje con la descripción del problema.

In [ ]:
# Convertir el DataFrame de Spark a un DataFrame de pandas
try:
    data_pd = data_v2.toPandas()
    print('DataFrame de Spark convertido exitosamente a DataFrame de pandas.')
    print('Mostrando las primeras filas del DataFrame de pandas:')
    display(data_pd.head())
    print(f'Número de registros en el DataFrame de pandas: {len(data_pd)}')
except Exception as e:
    print(f'Ocurrió un error al convertir el DataFrame a pandas: {e}')

Exportamos el `DataFrame` de pandas `data_pd` a un archivo `CSV` en la ruta especificada dentro de Google Drive. El parámetro `index=False` indica que no se incluirá la columna de índices en el archivo exportado, manteniendo solo los datos originales en el `CSV`.

In [ ]:
data_pd.to_csv('/content/drive/MyDrive/TSCDeIA - Práctica Profesionalizante/Procesamiento de Datos/data.csv', index=False)